In [1]:
from typing import Tuple
import pandas as pd
from scipy.stats import pearsonr
from tqdm.notebook import tqdm
import sys
sys.path.append('..')
from evaluation.generated_dataset import GeneratedDataset, load_all_from_config
from evaluation.novelty import NoveltyFilter, filter_by_unique_structure, filter_by_unique_structure_chem_sys_index

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


In [2]:
dft_datasets = {
    "WyFormerDirect": ("WyckoffTransformer", "DFT"),
    "WyFormerCrySPR": ("WyckoffTransformer", "CrySPR", "CHGNet_fix", "DFT"),
    "WyFormerDiffCSP++": ("WyckoffTransformer", "DiffCSP++", "DFT"),
    "WyFormerDiffCSP++10k": ("WyckoffTransformer", "DiffCSP++10k", "CHGNet_free", "DFT"),
    "WyFormerDiffCSP++10k-GGA-relax-1": ("WyckoffTransformer", "DiffCSP++10k", "CHGNet_free", "DFT-GGA-realax-1"),
    "WyFormerHarmonicDiffCSP++": ("WyckoffTransformer-harmonic", "DiffCSP++", "DFT"),
    "WyLLM-DiffCSP++": ("WyckoffLLM-naive", "DiffCSP++", "DFT"),
    "WyFormer-letters-DiffCSP++": ("WyckoffTransformer-letters", "DiffCSP++", "DFT"),
    "SymmCD": ("SymmCD", "DFT"),
    "DiffCSP": ("DiffCSP", "DFT"),
    "CrystalFormer": ("CrystalFormer", "DFT"),
    "DiffCSP++": ("DiffCSP++", "DFT"),
    "FlowMM": ("FlowMM", "DFT")
}

source_datasets = {name: t[:-1] for name, t in dft_datasets.items()}

In [3]:
chgnet_datasets = {
    "WyFormerDirect": ("WyckoffTransformer", "CrySPR", "CHGNet_fix_release"),
    "WyFormerCrySPR": ("WyckoffTransformer", "CrySPR", "CHGNet_fix_release"),
    "WyFormerDiffCSP++": ("WyckoffTransformer", "DiffCSP++", "CHGNet_fix"),
    "WyFormerDiffCSP++10k": ("WyckoffTransformer", "DiffCSP++10k", "CHGNet_free"),
    "WyLLM-DiffCSP++": ("WyckoffLLM-naive", "DiffCSP++", "CHGNet_fix"),
    "WyFormer-letters-DiffCSP++": ("WyckoffTransformer-letters", "DiffCSP++", "CHGNet_fix"),
    "WyFormerHarmonicDiffCSP++": ("WyckoffTransformer-harmonic", "DiffCSP++", "CHGNet_fix"),
    "SymmCD": ("SymmCD", "CHGNet_fix"),
    "DiffCSP": ("DiffCSP", "CHGNet_fix"),
    "CrystalFormer": ("CrystalFormer", "CHGNet_fix_release"),
    "DiffCSP++": ("DiffCSP++", "CHGNet_fix_release"),
    "FlowMM": ("FlowMM", "CHGNet_fix")
}

In [4]:
chgnet_data = load_all_from_config(datasets=list(chgnet_datasets.values()) + [('WyckoffTransformer', 'CrySPR', 'CHGNet_fix')])

In [5]:
all_datasets = load_all_from_config(
    datasets=list(dft_datasets.values()) + list(source_datasets.values()) + \
        [("split", "train"), ("split", "val"), ("split", "test")],
    dataset_name="mp_20")

In [6]:
wycryst_transformations = ('WyCryst', 'CrySPR', 'CHGNet_fix')
source_datasets["WyCryst"] = wycryst_transformations
chgnet_datasets["WyCryst"] = wycryst_transformations
chgnet_data[wycryst_transformations] = GeneratedDataset.from_cache(wycryst_transformations, "mp_20_biternary")
dft_datasets["WyCryst"] = tuple(list(wycryst_transformations) + ["DFT"])
all_datasets[dft_datasets["WyCryst"]] = GeneratedDataset.from_cache(dft_datasets["WyCryst"], "mp_20_biternary")

In [7]:
excluded_categories = frozenset(["radioactive", "rare_earth_metal", "noble_gas"])
from pymatgen.core import Structure
def check_composition(structure: Structure) -> bool:
    for category in excluded_categories:
        if structure.composition.contains_element_type(category):
            return False
    return True

In [8]:
novelty_reference = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
novelty_filter = NoveltyFilter(novelty_reference)

In [9]:
import evaluation.statistical_evaluator
test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(all_datasets[('split', 'test')].data)

In [10]:
import evaluation.novelty
train_w_template_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1))

In [11]:
def is_sg_preserved(relaxed_sg, transformations: Tuple[str]) -> pd.Series:
    source_sg = all_datasets[transformations[:-1]].data.spacegroup_number
    return relaxed_sg == source_sg.reindex_like(relaxed_sg)

In [12]:
mp_20 = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'test')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
(mp_20.spacegroup_number == 1).mean()
mp_20.smact_validity.mean()

0.9057020937893829

In [13]:
from collections import Counter
from operator import itemgetter
from itertools import chain
element_counts = Counter(chain(*mp_20.elements))

In [14]:
represented_elements=frozenset(map(itemgetter(0), element_counts.most_common(30)))

In [15]:
def check_represented_composition(structure: Structure) -> bool:
    for element in structure.composition:
        if element not in represented_elements:
            return False
    return True

In [16]:
top_10_groups = frozenset(mp_20.spacegroup_number.value_counts().iloc[:10].index)
n_elements_dist = {}

In [17]:
train_novelty_filter = NoveltyFilter(all_datasets[('split', 'train')].data)

In [18]:
from evaluation.novelty import filter_by_unique_structure_chem_sys_index

In [19]:
def get_train_based_sun(dataset: pd.DataFrame, intial_count: int):
    unique = filter_by_unique_structure_chem_sys_index(dataset)
    print("Unique structures %", len(unique)/len(dataset) * 100)
    unique_novel = train_novelty_filter.get_novel(unique)
    print("Novel & Unique structures %", len(unique_novel)/len(dataset) * 100)
    sun = ((unique_novel.e_above_hull_corrected < 0) & unique_novel.structure.map(lambda s: len(set(s.composition)) >=2)).sum()
    print("S.U.N. % (not counting failed)", sun/len(dataset) * 100)
    print("S.U.N. % (counting failed)", sun/intial_count * 100)

In [22]:
get_train_based_sun(all_datasets[dft_datasets["WyFormerDiffCSP++10k"]].data, 1e4)

Unique structures % 98.80494648238594
Novel & Unique structures % 92.60105996051128
S.U.N. % (not counting failed) 4.115140808479684
S.U.N. % (counting failed) 3.9600000000000004


In [23]:
get_train_based_sun(all_datasets[dft_datasets["WyFormerDiffCSP++10k-GGA-relax-1"]].data, 1e4)

Unique structures % 98.80855397148676
Novel & Unique structures % 92.60692464358452
S.U.N. % (not counting failed) 4.429735234215886
S.U.N. % (counting failed) 4.35


In [20]:
novel = train_novelty_filter.get_novel(wyformer_10k)
print(len(filter_by_unique_structure_chem_sys_index(novel))/len(novel))

0.965513536742587


In [21]:
len(unique)/10000

0.9585

In [22]:
wyformer_10k["is_novel"] = wyformer_10k.apply(novelty_filter.is_novel, axis=1)
wyformer_10k["is_novel_train"] = wyformer_10k.apply(train_novelty_filter.is_novel, axis=1)

In [23]:
wyformer_10k["is_novel"].mean()

0.9142341257899489

In [24]:
wyformer_10k["is_novel_train"].mean()

0.9336944528036915

In [25]:
(wyformer_10k.e_above_hull_corrected < 0).sum()/10000

0.0535

In [26]:
(wyformer_10k.e_above_hull_corrected < 0.1).sum()/10000

0.3171

Validity
1. Vanilla; Valid records: 2866 / 9648 = 29.71%
2. Naive; Valid records: 9492 / 9804 = 96.82%
3. Site Symmetry; Valid records: 8955 / 9709 = 92.23%

In [27]:
tables = {}
for E_hull_threshold in (0, 0.08):
    table = pd.DataFrame(
        index=dft_datasets.keys(), columns=[
            "DFT dataset size",
            "Source Novelty (%)",
            "In-DFT Novelty (%)",
            "S.U.N. (%)",
            "P1 in source (%)",
            "S.S.U.N. (%)"])
    table.index.name = "Method"

    for name, transformations in tqdm(dft_datasets.items()):
        dataset = all_datasets[transformations]
        if "corrected_e_hull" not in dataset.data.columns:
            dataset.data["corrected_e_hull"] = dataset.data.e_above_hull_corrected
        table.loc[name, "DFT dataset size"] = len(dataset.data)
        try:
            source_dataset = all_datasets[transformations[:-1]]
        except KeyError:
            source_dataset = chgnet_data[transformations[:-1]]
        chgnet_dataset = chgnet_data[chgnet_datasets[name]]

        unique = filter_by_unique_structure(dataset.data)
        novel = novelty_filter.get_novel(unique)
        table.loc[name, "In-DFT Novelty (%)"] = 100 * len(novel) / len(unique)
        source_novel = novelty_filter.get_novel(source_dataset.data)
        source_novelty = 100 * len(source_novel) / len(source_dataset.data)
        table.loc[name, "Source Novelty (%)"] = len(novel) / len(unique) * source_novelty
        table.loc[name, "P1 in source (%)"] = 100 * (source_novel.group == 1).mean()
        try:
            table.loc[name, "SG preserved (%)"] = 100 * is_sg_preserved(novel.spacegroup_number, transformations).mean()
        except KeyError:
            pass
        # source_novel_symmetric = (source_novel.group != 1).sum() / len(source_dataset.data)
        # table["Source Novel !P1 (%)"] = 100 * source_novel_symmetric
        # DFT failure == unreal structure
        if name == "WyFormerDiffCSP++10k":
            # This dataset is a bit special, as it contains 10k structures from the WyckoffTransformer
            # not filtered by the novelty filter
            dft_structures = 10000
            source_novelty = 100
        else:
            dft_structures = 105
        has_ehull = dataset.data.corrected_e_hull.notna()
        is_sun = (novel.corrected_e_hull <= E_hull_threshold) # & (novel.elements.apply(lambda x: len(frozenset(x))) >= 2)
        table.loc[name, "S.U.N. (%)"] = source_novelty * is_sun.sum() / dft_structures
        table.loc[name, "total_sun"] = is_sun.sum().astype(int)
        table.loc[name, "S.S.U.N. (%)"] = source_novelty * (is_sun & (novel.group != 1)).sum() / dft_structures
        table.loc[name, "total_ssun"] = (is_sun & (novel.group != 1)).sum().astype(int)
        table.loc[name, "P1 in stable (%)"] = 100 * (novel[is_sun].group == 1).mean()

        chgnet_unique = filter_by_unique_structure(chgnet_dataset.data)
        chgnet_novel = novelty_filter.get_novel(chgnet_unique)
        if "corrected_chgnet_ehull" in chgnet_dataset.data.columns:
            chgnet_is_sun = (chgnet_novel.corrected_chgnet_ehull < E_hull_threshold)
            #table.loc[name, "CHGNet dataset size"] = chgnet_dataset.data.corrected_chgnet_ehull.notna().sum()
            table.loc[name, "S.U.N. (CHGNet) (%)"] =  100 * chgnet_is_sun.sum() / chgnet_dataset.data.corrected_chgnet_ehull.notna().sum()
            table.loc[name, "S.S.U.N. (CHGNet) (%)"] = 100 * (chgnet_is_sun & (chgnet_novel.group != 1)).sum() / chgnet_dataset.data.corrected_chgnet_ehull.notna().sum()
            
            chgnet_dft_available = chgnet_dataset.data.reindex(dataset.data.index[has_ehull])
            table.loc[name, "r DFT CHGNet"] = \
                pearsonr((chgnet_dft_available.corrected_chgnet_ehull < E_hull_threshold).astype(float),
                        (dataset.data.corrected_e_hull < E_hull_threshold).astype(float)).correlation
    tables[E_hull_threshold] = table

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [28]:
tables[0]

,DFT dataset size,Source Novelty (%),In-DFT Novelty (%),S.U.N. (%),P1 in source (%),S.S.U.N. (%),SG preserved (%),total_sun,total_ssun,P1 in stable (%),S.U.N. (CHGNet) (%),S.S.U.N. (CHGNet) (%),r DFT CHGNet
Method,,,,,,,,,,,,,
WyFormerDirect,94,90.09,100.0,0.858,1.964702,0.858,86.170213,1.0,1.0,0.00000,13.813814,13.713714,0.206021
WyFormerCrySPR,104,90.0,100.0,4.285714,1.555556,4.285714,96.153846,5.0,5.0,0.00000,13.813814,13.713714,0.323818
WyFormerDiffCSP++,104,88.639423,99.038462,4.261905,1.564246,4.261905,95.145631,5.0,5.0,0.00000,12.400000,12.400000,0.334818
WyFormerDiffCSP++10k,9969,84.207425,91.95127,3.66,6.968108,3.61,90.329521,366.0,361.0,1.36612,NaN,NaN,NaN
WyFormerHarmonicDiffCSP++,101,92.6,100.0,4.409524,2.37581,4.409524,92.079208,5.0,5.0,0.00000,8.341811,8.240081,0.359760
WyLLM-DiffCSP++,102,94.578313,100.0,4.503729,1.380042,4.503729,99.019608,5.0,5.0,0.00000,7.849134,7.747197,0.778248
WyFormer-letters-DiffCSP++,104,90.562249,100.0,3.44999,1.108647,3.44999,96.153846,4.0,4.0,0.00000,8.654849,8.654849,0.529150
SymmCD,96,90.649077,100.0,5.179947,2.177203,5.179947,92.708333,6.0,6.0,0.00000,10.365344,10.344468,-0.097590
DiffCSP,104,88.082885,98.076923,6.842667,31.566641,5.987333,78.431373,8.0,7.0,12.50000,17.800000,15.100000,0.441401


In [29]:
tables[0.08]

,DFT dataset size,Source Novelty (%),In-DFT Novelty (%),S.U.N. (%),P1 in source (%),S.S.U.N. (%),SG preserved (%),total_sun,total_ssun,P1 in stable (%),S.U.N. (CHGNet) (%),S.S.U.N. (CHGNet) (%),r DFT CHGNet
Method,,,,,,,,,,,,,
WyFormerDirect,94,90.09,100.0,4.29,1.964702,4.29,86.170213,5.0,5.0,0.000000,39.239239,38.238238,0.269486
WyFormerCrySPR,104,90.0,100.0,23.142857,1.555556,22.285714,96.153846,27.0,26.0,3.703704,39.239239,38.238238,0.664923
WyFormerDiffCSP++,104,88.639423,99.038462,22.161905,1.564246,21.309524,95.145631,26.0,25.0,3.846154,36.700000,36.000000,0.615387
WyFormerDiffCSP++10k,9969,84.207425,91.95127,18.34,6.968108,17.93,90.329521,1834.0,1793.0,2.235551,NaN,NaN,NaN
WyFormerHarmonicDiffCSP++,101,92.6,100.0,20.28381,2.37581,20.28381,92.079208,23.0,23.0,0.000000,35.503561,34.486267,0.593771
WyLLM-DiffCSP++,102,94.578313,100.0,11.709696,1.380042,11.709696,99.019608,13.0,13.0,0.000000,31.600408,30.886850,0.592083
WyFormer-letters-DiffCSP++,104,90.562249,100.0,18.974947,1.108647,18.974947,96.153846,22.0,22.0,0.000000,31.491137,30.969760,0.548767
SymmCD,96,90.649077,100.0,20.719789,2.177203,20.719789,92.708333,24.0,24.0,0.000000,34.551148,34.070981,0.108845
DiffCSP,104,88.082885,98.076923,22.238667,31.566641,20.528,78.431373,26.0,24.0,7.692308,57.400000,40.600000,0.405781


In [30]:
def prettify(table):
    return table.style.format({
    "S.U.N. (%)": "{:.1f}",
    "S.S.U.N. (%)": "{:.1f}",
    #"S.U.N. (CHGNet) (%)": "{:.1f}",
    #"S.S.U.N. (CHGNet) (%)": "{:.1f}",
    #"r DFT CHGNet": "{:.2f}",
}).highlight_max(props="font-weight: bold", axis=0, subset=["S.U.N. (%)", "S.S.U.N. (%)"])

In [31]:
selected_table = table.loc[:, ["S.U.N. (%)", "S.S.U.N. (%)"]]

In [32]:
pretty_table = prettify(selected_table)
pretty_table.to_latex("tables/dft.tex", siunitx=True, convert_css=True)
pretty_table

,S.U.N. (%),S.S.U.N. (%)
Method,,
WyFormerDirect,4.3,4.3
WyFormerCrySPR,23.1,22.3
WyFormerDiffCSP++,22.2,21.3
WyFormerDiffCSP++10k,18.3,17.9
WyFormerHarmonicDiffCSP++,20.3,20.3
WyLLM-DiffCSP++,11.7,11.7
WyFormer-letters-DiffCSP++,19.0,19.0
SymmCD,20.7,20.7
DiffCSP,22.2,20.5


In [33]:
all_datasets[('split', 'test')].data.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1).isin(train_w_template_set).mean()

0.9713685606898077

In [ ]:
from scipy.stats import ttest_ind
import numpy as np
def get_observation(name, column="total_ssun"):
    all_observations = np.zeros(dft_structures)
    all_observations[:int(table.at[name, column])] = table.loc[name, "Source Novelty (%)"]/100
    return all_observations

In [ ]:
for second in table.index:
    print(second, ttest_ind(get_observation("FlowMM"), get_observation(second)))

WyFormerDirect TtestResult(statistic=2.8699643276771534, pvalue=0.004529164843545496, df=208.0)
WyFormerCrySPR TtestResult(statistic=2.0489237904086535, pvalue=0.04172504759818785, df=208.0)
WyFormerDiffCSP++ TtestResult(statistic=0.5399251990582908, pvalue=0.5898261986038633, df=208.0)
DiffCSP TtestResult(statistic=0.3358706725126662, pvalue=0.7373069464508927, df=208.0)
CrystalFormer TtestResult(statistic=0.4278113819160564, pvalue=0.669231052265232, df=208.0)
DiffCSP++ TtestResult(statistic=1.8300352833914524, pvalue=0.0686759122341314, df=208.0)
FlowMM TtestResult(statistic=0.0, pvalue=1.0, df=208.0)


In [ ]:
for second in table.index:
    print(second, ttest_ind(get_observation("DiffCSP", column="total_sun"), get_observation(second, column="total_sun")))

WyFormerDirect TtestResult(statistic=3.7329421055392644, pvalue=0.0002442600601858129, df=208.0)
WyFormerCrySPR TtestResult(statistic=2.933952952207651, pvalue=0.0037223172815396767, df=208.0)
WyFormerDiffCSP++ TtestResult(statistic=1.444489793625256, pvalue=0.15010593136692968, df=208.0)
DiffCSP TtestResult(statistic=0.0, pvalue=1.0, df=208.0)
CrystalFormer TtestResult(statistic=1.3621411208732088, pvalue=0.1746266271124153, df=208.0)
DiffCSP++ TtestResult(statistic=2.7203519120538746, pvalue=0.007073135428130352, df=208.0)
FlowMM TtestResult(statistic=0.7296066705938044, pvalue=0.4664514752444603, df=208.0)
